## 1 Settings

### 1) Import required libraries

In [85]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [86]:
import unidecode
import string
import random
import re
import time, math

## Hyperparameter

In [87]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150  
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

## 2. Data
### 1) Prepare characters

In [88]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 2) Get text data

In [89]:
file = unidecode.unidecode(open('./data/shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


## 3. Functions for text processing
### 1) Random Chunk

In [90]:
def random_chunk(): ## 세익스피어 뭉치에서 chunk_len 200만큼 랜덤으로 텍스트 추출
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())


ou livest and breathest,
Yet art thou slain in him: thou dost consent
In some large measure to thy father's death,
In that thou seest thy wretched brother die,
Who was the model of thy father's life.
C


### 2) Character to tensor

In [91]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('yunsikus'))

tensor([34, 30, 23, 28, 18, 20, 30, 28])


In [92]:
all_characters.index('yunsiks'[0]) ## y의 인덱스가 34

34

### 3) Chink into input & label

In [93]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [94]:
inp,target = random_training_set()

In [95]:
inp

tensor([11, 21, 14, 77, 96, 37, 14, 21, 18, 20, 14, 94, 34, 24, 30, 94, 22, 14,
        10, 23, 94, 29, 24, 94, 22, 10, 20, 14, 94, 10, 94, 25, 30, 25, 25, 14,
        29, 94, 24, 15, 94, 22, 14, 75, 96, 96, 51, 40, 55, 53, 56, 38, 43, 44,
        50, 77, 96, 58, 17, 34, 73, 94, 29, 27, 30, 14, 78, 94, 17, 14, 94, 22,
        14, 10, 23, 28, 94, 29, 24, 94, 22, 10, 20, 14, 94, 10, 94, 25, 30, 25,
        25, 14, 29, 94, 24, 15, 94, 29, 17, 14, 14, 75, 96, 96, 55, 10, 18, 21,
        24, 27, 77, 96, 54, 17, 14, 94, 28, 10, 34, 28, 94, 34, 24, 30, 27, 94,
        32, 24, 27, 28, 17, 18, 25, 94, 22, 14, 10, 23, 28, 94, 29, 24, 94, 22,
        10, 20, 14, 96, 10, 94, 25, 30, 25, 25, 14, 29, 94, 24, 15, 94, 17, 14,
        27, 75, 96, 96, 51, 40, 55, 53, 56, 38, 43, 44, 50, 77, 96, 50, 94, 22,
        24, 23, 28, 29, 27, 24, 30, 28, 94, 10, 27, 27, 24, 16, 10, 23, 12, 14,
        62, 94])

In [96]:
target

tensor([21, 14, 77, 96, 37, 14, 21, 18, 20, 14, 94, 34, 24, 30, 94, 22, 14, 10,
        23, 94, 29, 24, 94, 22, 10, 20, 14, 94, 10, 94, 25, 30, 25, 25, 14, 29,
        94, 24, 15, 94, 22, 14, 75, 96, 96, 51, 40, 55, 53, 56, 38, 43, 44, 50,
        77, 96, 58, 17, 34, 73, 94, 29, 27, 30, 14, 78, 94, 17, 14, 94, 22, 14,
        10, 23, 28, 94, 29, 24, 94, 22, 10, 20, 14, 94, 10, 94, 25, 30, 25, 25,
        14, 29, 94, 24, 15, 94, 29, 17, 14, 14, 75, 96, 96, 55, 10, 18, 21, 24,
        27, 77, 96, 54, 17, 14, 94, 28, 10, 34, 28, 94, 34, 24, 30, 27, 94, 32,
        24, 27, 28, 17, 18, 25, 94, 22, 14, 10, 23, 28, 94, 29, 24, 94, 22, 10,
        20, 14, 96, 10, 94, 25, 30, 25, 25, 14, 29, 94, 24, 15, 94, 17, 14, 27,
        75, 96, 96, 51, 40, 55, 53, 56, 38, 43, 44, 50, 77, 96, 50, 94, 22, 24,
        23, 28, 29, 27, 24, 30, 28, 94, 10, 27, 27, 24, 16, 10, 23, 12, 14, 62,
        94, 55])

## 3. Model & Optimizer

### 1) Model

In [138]:
class RNN(nn.Module):
    def __init__(self,input_size,embedding_size,hidden_size,output_size,num_layers=1):
        super(RNN,self).__init__()
        self.input_size = input_size ## 35
        self.hidden_size = hidden_size ## 100
        self.output_size = output_size
        self.num_layers= num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out, hidden = self.rnn(out, hidden)
        out = self.decoder(out.view(batch_size,-1))
        
        return out, hidden
    
    def init_hidden(self):
        hidden= Variable(torch.zeros(self.num_layers, batch_size, hidden_size))
        return hidden
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers=1)
        ## 2번쨰 n_characters는 outputsize랑 같다

In [139]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())  ## num_layers, batch_size, hidden_size

out, hidden = model(inp,hidden)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


### 2) Loss & Optimizer

In [140]:
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()

### 3) Test function

In [141]:
start_str = "b" ## b로 시작 이것도 확률분포로 만들 수 있지 않을까?
inp = char_tensor(start_str)
hidden = model.init_hidden()
x = inp
for i in range(1):
    output, hidden = model(x,hidden)
    output_dist = output.data.view(-1).div(0.8).exp()    
    top_i = torch.multinomial(output_dist,1)[0]
    print(top_i)

tensor(59)


In [142]:
x.size()

torch.Size([1])

In [143]:
output_dist

tensor([0.7439, 1.5146, 1.1236, 0.7769, 2.1880, 0.6666, 1.4492, 0.5392, 0.9635,
        0.7870, 1.3467, 0.8468, 0.7555, 0.8428, 0.7420, 1.4124, 0.7087, 1.8945,
        1.6752, 0.6588, 0.6291, 0.8761, 1.1234, 1.6576, 1.1545, 0.9888, 1.4553,
        0.8164, 0.8039, 0.8709, 1.2871, 0.8534, 2.0909, 0.6292, 0.6667, 0.7503,
        1.0221, 1.1365, 0.4136, 0.8479, 0.6134, 0.8297, 0.9006, 0.8570, 0.8674,
        1.6810, 1.1234, 1.7378, 0.6311, 0.8464, 1.3804, 0.4324, 0.8817, 0.6774,
        0.6859, 0.7459, 0.5737, 0.6090, 0.7238, 0.9889, 0.5957, 1.7250, 0.9680,
        1.3992, 0.9207, 0.5543, 0.7294, 0.8411, 0.9374, 1.4614, 0.5707, 0.6041,
        0.7772, 1.0397, 0.6279, 1.0685, 1.0852, 1.7853, 0.8995, 1.4530, 1.2192,
        1.1256, 1.6676, 0.6446, 1.2014, 0.9949, 0.4919, 1.2112, 0.8351, 0.7915,
        1.1694, 1.0610, 1.0368, 1.5040, 1.0098, 1.4726, 1.6508, 0.8255, 0.6520,
        0.8373])

In [145]:
for epoch in range(1):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()
    
    loss = 0
    optimizer.zero_grad()
    for i in range(1):
        x = inp[i]
        y_ = label[i]
        y,hidden = model(x,hidden)


In [153]:
x.size()

torch.Size([])

In [154]:
y.size()

torch.Size([1, 100])

In [158]:
y_.view(-1).size()

torch.Size([1])

In [149]:
hidden

tensor([[[-0.1408,  0.7188,  0.5278, -0.0148, -0.5549, -0.8979, -0.5828,
          -0.5003, -0.7104,  0.0867,  0.4030, -0.0624,  0.5574,  0.1014,
          -0.2588, -0.0042,  0.4902, -0.3436,  0.5033, -0.2718, -0.4009,
          -0.3011,  0.2921, -0.4843,  0.6959,  0.8544,  0.0139, -0.0100,
           0.1575, -0.7728,  0.1670, -0.2857, -0.0505, -0.7432, -0.1427,
          -0.6114, -0.8402,  0.1639, -0.3281,  0.4113, -0.4486,  0.9151,
          -0.4658, -0.4356, -0.8988,  0.5342,  0.3038, -0.6788, -0.0262,
          -0.7841,  0.8736, -0.0173, -0.7930,  0.1247,  0.8178, -0.6351,
           0.7127,  0.3769,  0.5729,  0.4580, -0.8677, -0.4816,  0.1188,
          -0.9233,  0.7388, -0.2918,  0.1238, -0.2786, -0.5910,  0.2711,
          -0.8166,  0.7907,  0.8036, -0.2333,  0.0796,  0.4989, -0.5476,
          -0.3519, -0.6112, -0.8517, -0.1181, -0.4000,  0.6478,  0.2811,
          -0.3273, -0.8308,  0.0569, -0.1781,  0.7788,  0.2830, -0.1876,
           0.8917, -0.3329, -0.7990,  0.8055,  0.64

In [150]:
y.size()

torch.Size([1, 100])

In [151]:
y_

tensor(22)

In [159]:
for epoch in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()
    
    loss = 0
    optimizer.zero_grad()
    for i in range(chunk_len-1):
        x = inp[i]
        y_ = label[i]
        y,hidden = model(x,hidden)
        loss += criterion(y,y_.view(-1)) ## y_의 size가 0이 되면 안된다. 
        
    loss.backward()
    optimizer.step()
    
    if epoch % 100 ==0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")
        
    


 tensor(4.6694, grad_fn=<DivBackward0>) 

b28EJp3\]v"7~ki_w1nDqCW\|Ny<EC%CBj>hSZ}pH0'RxBZ7bl.x2!(?p(_l} .pAO# zTtJbe#\="Y|uFV>I,U6A4iZqIMMF5zD{V	M`[-%%NQj;&3@&oJ=`h&:!CX^Q$~'x:&zycNu*TNP+[mT`JCd6,^7`;R{P!ccGcWnB~3y[~`|gn86!TM[Y:CLcm\e na7!zE



 tensor(2.4487, grad_fn=<DivBackward0>) 

boud tamnd, filly hat hars, te ceot be weth pfndt oist oou? gart t thepis, serss, he' yse tnen ho, sit ond the thar he tsenceaof th th men, se soram oo Ren tors owetry sises, vit le he done cher ouan s



 tensor(2.1592, grad_fn=<DivBackward0>) 

brithe nate on.

Sent ifeber thald Iup bent Wid tor hath thard the bred no higm my urther'd the of ye I wad not med at 'the kit, hor ife of bin hear ofy
Tham wiles mesit in sreancelyeW, that brit the m



 tensor(2.1907, grad_fn=<DivBackward0>) 

bly! thur wheis you mum prears his nowe derding mymenacin monted for hit oiter my do ford will all youd sthes Gors.

BY
And I speerve?

IOLANUCACANUS:
Lut fravald serke havenath'ded haste me hest to th



 tensor(2.1

Bear the earpe of the pond soy me of is not lies, and is h



 tensor(1.8836, grad_fn=<DivBackward0>) 

be not poulse thee a fainst not him time as thy holil, and lild did the tous on trus my nowlll thou, and and that thou so.

PETRUCHIO:
Where live to good deging hanl to the bro, with I hands more thing



 tensor(1.9814, grad_fn=<DivBackward0>) 

bit slould the wid sicks tring hould deat of a contong the come.

RORTIS:
She hank'd but to thou may here notst thou with morrook your weantle mainty to she hing hall a hus trust to not Farnd panst sha



 tensor(1.4662, grad_fn=<DivBackward0>) 

ber son Parther he the so men to need comest of me sope retite the loteetire this wingle,
And andward from you, with could, to more will that this made.

KINCENTIO: more in we us is unfuch the hink the



 tensor(1.7729, grad_fn=<DivBackward0>) 

best the with not him hand;
What and shall the spe consulsem that on that lice bethine and for not not the yeant do-night to
lay.

DUKE VINCENTIO:
Plingsh 

성능이 별로 안좋다 LSTM과 GRU도 해보자